In [1]:
from Bert import Bert
import torch
from tokenizers.implementations import BertWordPieceTokenizer

In [2]:
_embedding_dim = 384
_hidden_size = 3072
_num_head = 12
_num_layers = 12
_tokenizer = BertWordPieceTokenizer("custom/vocab.txt")

In [8]:
bert = Bert(_embedding_dim, _hidden_size, _num_head, 128, _num_layers, _tokenizer, device='cpu')

<All keys matched successfully>

In [9]:
bert.predict_mask('He is [MASK] that he [MASK] the team to [MASK].')

{0: 'convinced', 1: 'wants', 2: 'win'}

<p style="line-height:1.2;">
原句：He is glad that he helped the team to win.<br>
翻译：他很高兴他帮助球队赢了<br>
预测：他确信他希望球队获胜<br>
</p>

In [10]:
bert.predict_mask('May there be [MASK] clouds in [MASK] life to make a [MASK] [MASK].')

{0: 'enough', 1: 'my', 2: 'new', 3: 'life'}

<p style="line-height:1.2;">
原句：May there be enough clouds in your life to make a beautiful sunset.<br>
翻译：愿你的生命中有足够的云翳，来造成一个美丽的黄昏<br>
预测：愿我的生命中有足够的云翳来创造新的生活<br>
</p>

In [11]:
bert.predict_mask('The [MASK] way to miss [MASK] is to be sitting right beside them knowing you [MASK] have them.')

{0: 'only', 1: 'them', 2: 'can'}

<p style="line-height:1.2;">
原句：The worst way to miss someone is to be sitting right beside them knowing you can’t have them<br>
翻译：失去某人，最糟糕的莫过于，他近在身旁，却犹如远在天边<br>
预测：想念他们的唯一方式，就是坐在他们身旁，知道你可以拥有他们<br>
</p>

In [13]:
import netron
import torch.onnx

In [15]:
text = bert.tokenizer.encode('The [MASK] way to miss [MASK] is to be sitting right beside them knowing you [MASK] have them.').ids
text = torch.nn.functional.pad(torch.tensor(text, device=bert.device, dtype=torch.int16),
                               (0, bert.max_length - len(text)))
prob_mask = text == torch.tensor(bert.tokenizer.token_to_id('[MASK]'), device=bert.device,
                                 dtype=torch.int16)
mask0 = text != 0
modelData = "./demo.pth"
torch.onnx.export(bert, (text, mask0), modelData)
netron.start(modelData)

C:\Users\123\PycharmProjects\torch-models\Bert.py:49: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  pad = torch.eq(mask0, torch.tensor(False, device=self.device, dtype=torch.bool))


======= Diagnostic Run torch.onnx.export version 2.1.0.dev20230519+cu121 =======
verbose: False, log level: 40
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Serving './demo.pth' at http://localhost:8080


('localhost', 8080)